#  Welcome to Space Application Part 4

## Poliastro

First the modules

In [1]:
from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

In [2]:
r = [-6045, -3490, 2500] * u.km
v = [-3.457, 6.618, 2.533] * u.km / u.s

orb = Orbit.from_vectors(Earth, r, v)

In [3]:
orb

7283 x 10293 km x 153.2 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)

In [4]:
orb.epoch

<Time object: scale='tt' format='jyear_str' value=J2000.000>

# Let's go to Mars

In [5]:
from astropy import units as u
from astropy import time

import numpy as np

from poliastro import iod
from poliastro.bodies import Earth, Mars, Sun
from poliastro.ephem import Ephem
from poliastro.maneuver import Maneuver
from poliastro.twobody import Orbit
from poliastro.util import time_range
from astropy.coordinates import solar_system_ephemeris

solar_system_ephemeris.set("jpl")

timeout: The read operation timed out

In [ ]:
# Initial data as per https://solarsystem.nasa.gov/missions/curiosity-msl/in-depth/
date_launch = time.Time("2011-11-26 15:02", scale="utc").tdb
date_arrival = time.Time("2012-08-06 05:17", scale="utc").tdb

In [ ]:
#Trajectories of Earth and Mars
earth = Ephem.from_body(Earth, time_range(date_launch, end=date_arrival))
mars = Ephem.from_body(Mars, time_range(date_launch, end=date_arrival))

In [ ]:
mars

In [ ]:
# Solve for departure and target orbits
ss_earth = Orbit.from_ephem(Sun, earth, date_launch)
ss_mars = Orbit.from_ephem(Sun, mars, date_arrival)

In [ ]:
ss_mars

We can now solve for the maneuver that will take us from Earth to Mars. After solving it, we just need to apply it to the departure orbit to solve for the transfer one:

In [ ]:
# Solve for the transfer maneuver
man_lambert = Maneuver.lambert(ss_earth, ss_mars)

# Get the transfer and final orbits
ss_trans, ss_target = ss_earth.apply_maneuver(man_lambert, intermediate=True)

In [ ]:
ss_trans

In [ ]:
ss_target

Let’s plot this transfer orbit in 3D!

In [ ]:
from poliastro.plotting import OrbitPlotter3D

In [ ]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth, date_launch, label="Earth at launch position",color="navy")
plotter.plot_ephem(Ephem.from_body(Earth, date_arrival), date_arrival, label="Earth at arrival position",color="navy")
plotter.plot_ephem(mars, date_arrival, label="Mars at arrival position",color="red")
plotter.plot_trajectory(ss_trans.sample(max_anomaly=180 * u.deg), color="black", label="Transfer orbit")

In [ ]:
from ipywidgets import interact
from plotly.graph_objects import FigureWidget
from poliastro.twobody.propagation import propagate

In [ ]:
def go_to_mars(tof_h):
    date_launch = time.Time("2011-11-26 15:02", scale="utc").tdb
#     date_launch+=offset
    date_arrival = date_launch + tof_h * u.h 
    mars_arrival = Ephem.from_body(Mars, time_range(date_launch, end=date_arrival))
    earth_arrival = Ephem.from_body(Earth, time_range(date_launch, end=date_arrival))
    ss_mars_arrival= Orbit.from_ephem(Sun, mars_arrival, date_arrival)
    ss_earth_arrival = Orbit.from_ephem(Sun, earth_arrival, date_launch)
    
    man_lambert = Maneuver.lambert(ss_earth_arrival, ss_mars_arrival)
    # Get the transfer orbit
    ss_trans, _ = ss_earth.apply_maneuver(man_lambert, intermediate=True)

    tofs = time.TimeDelta(np.linspace(0 * u.h, tof_h * u.h, num=100))
    ss_trans_traj = propagate(
        ss_trans,
        tofs,
    )

    with fig.batch_update():
        for trace in fig.data:
            trace.visible = False
        plotter.set_attractor(Sun)
        plotter.plot_ephem(Ephem.from_body(Earth,date_launch), date_launch, label="Earth at launch position",color="navy")
        plotter.plot_ephem(Ephem.from_body(Earth, time_range(date_launch, end=date_arrival)),date_arrival, label="Earth at arrival position",color="navy")
        plotter.plot_ephem(mars_arrival,date_arrival, label="Mars at arrival position", color="red")
        plotter.plot_trajectory(ss_trans_traj, color="black", label="Transfer orbit")

In [ ]:
fig = FigureWidget()
plotter = OrbitPlotter3D(figure=fig)
plotter.set_attractor(Sun)
plotter.plot_ephem(Ephem.from_body(Earth, date_launch), date_launch, label="Earth at launch position",color="navy")
plotter.plot_ephem(Ephem.from_body(Earth, time_range(date_launch, end=date_arrival)),date_arrival, label="Earth at arrival position",color="navy")
# plotter.plot_body(earth, date_arrival, label="Earth at arrival position")
plotter.plot_ephem(mars, date_arrival, label="Mars at arrival position",color="red")
plotter.plot_trajectory(ss_trans.sample(max_anomaly=180 * u.deg), color="black", label="Transfer orbit")
plotter.set_view(30 * u.deg, 260 * u.deg, distance=2* u.km)


In [ ]:
interact(go_to_mars,tof_h=(100,12000))